In [11]:
using DataFrames, ScikitLearn, XLSX, DelimitedFiles

In [12]:
# Variables Globales
const PATH = pwd()
const AUX_FOLDER = "aux_file"
const OUTPUT = "output"
const MODEL_PARAMETERS = "model_weights.txt"
const OUTPUT_FILENAME = "predictions.xlsx"

"predictions.xlsx"

In [13]:
# Si no esta la carpeta output, la creo
if !isdir(joinpath(PATH, OUTPUT))
    mkdir(joinpath(PATH, OUTPUT))
end

In [14]:
new_data = Dict(
    "Fecha" => ["2021-01-01", "2021-02-01", "2021-03-01", "2021-04-01", "2021-05-01"],
    "Area" => [100, 200, 150, 120, 180],
    "Habitaciones" => [2, 3, 4, 3, 4],
    "Antiguedad" => [10, 20, 30, 40, 50]
)
new_df = DataFrame(new_data)

Row,Antiguedad,Area,Fecha,Habitaciones
,Int64,Int64,String,Int64
1,10,100,2021-01-01,2
2,20,200,2021-02-01,3
3,30,150,2021-03-01,4
4,40,120,2021-04-01,3
5,50,180,2021-05-01,4


In [15]:
media = [1, 2, 3, 4]
desv_estandar = [1, 2, 3, 4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [16]:
# Aplicar el mismo preprocesamiento que al dataset original
new_df_scaled = (new_df[:, [:Area, :Habitaciones, :Antiguedad]] .- media[1:end-1]') ./ desv_estandar[1:end-1]'

Row,Area,Habitaciones,Antiguedad
,Float64,Float64,Float64
1,99.0,0.0,2.33333
2,199.0,0.5,5.66667
3,149.0,1.0,9.0
4,119.0,0.5,12.3333
5,179.0,1.0,15.6667


In [17]:
# Ruta al archivo .txt
file_path = joinpath(PATH, AUX_FOLDER, MODEL_PARAMETERS)

# Cargar los coeficientes e intercept desde el archivo de texto
data = readdlm(file_path)
coeficientes = data[1:end-1]
intercept = data[end]

330337.0937274716

In [18]:
# Convertir el DataFrame en una matriz
m = Matrix(new_df_scaled)

# Realizar predicciones con el modelo entrenado
predictions = m * coeficientes .+ intercept

# Escalamiento de variable predecida
predictions = (predictions .* desv_estandar[end]) .+ media[end]


5-element Vector{Float64}:
 5.296454334623361e7
 1.0495551701305373e8
 7.791660685537073e7
 6.133043891756158e7
 9.224677689784755e7

In [19]:
# Agregar la columna de predicciones al DataFrame
new_df[!, :Predicciones] = predictions

5-element Vector{Float64}:
 5.296454334623361e7
 1.0495551701305373e8
 7.791660685537073e7
 6.133043891756158e7
 9.224677689784755e7

In [20]:
# Guardar las predicciones en un archivo de Excel
XLSX.writetable(
    joinpath(PATH, OUTPUT, OUTPUT_FILENAME),
    new_df,
    overwrite = true,
    sheetname = "Sheet1"
)